### ORBIT Dudgeon Wind Farm Validation

National Renewable Energy Lab

Matt Shields, Jake Nunemaker

Updated: 1/15/2020

In [36]:
import os
import sys
import pprint
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as plt_dates

import ORBIT
print(f"Using ORBIT version {ORBIT.__version__}.")
from ORBIT import ProjectManager

Using ORBIT version "0.3.2".


#### Weather File

In [37]:
filepath = os.path.join('library', 'weather', 'dudgeon_wave_100m_wind.csv')
weather = pd.read_csv(filepath).set_index(keys='datetime')
weather.describe()

,windspeed,waveheight
count,20616.000000,20616.000000
mean,9.138532,1.183350
std,4.249767,0.667603
min,0.121744,0.170372
25%,5.980583,0.698537
50%,8.916669,1.027665
75%,11.878717,1.507961
max,25.680700,5.773216


#### ORBIT Configuration

In [38]:
phases = [
    # Substructures
    'MonopileInstallation',
    'ScourProtectionDesign',
    'ScourProtectionInstallation',
    
    # Turbines
    'TurbineInstallation',
    
    # Electrical
    'ArraySystemDesign',
    'ArrayCableInstallation',
    'ExportSystemDesign',
    'ExportCableInstallation',
    'OffshoreSubstationInstallation'
]

required_config = ProjectManager.compile_input_dict(phases)
# required_config

In [4]:
config = {
    # Substations
    "num_substations": 1,
    
    # Vessels
    'scour_protection_install_vessel': 'ExampleScour',
    'trench_dig_vessel': 'StematSpirit',
    'array_cable_lay_vessel': 'Ndeavor',
    'export_cable_lay_vessel': 'StematSpirit',
    "oss_install_vessel": "OlegStrashnov",  # Actually vessel from SPT Offshore
    
    # Site/plant
    'site': {
        'depth': 22.5,
        'distance': 124,
        'distance_to_landfall': 35,
        'distance_to_beach': 0,
        'distance_to_interconnection': 3,
        'mean_windspeed': 9.13
    },
    
    'plant': {
        'layout': 'grid',
        'num_turbines': 67,
        'row_spacing': 7,
        'turbine_spacing': 1070 / 154,
        'substation_distance': 1
    },
    
    'port': {
        'num_cranes': 1,
        'monthly_rate': 2000000,
        "name": "Green Port"
    },
    
    # Turbine + components
    'turbine': '8MW_generic',

    # Substructure components
    'substructure': {'diameter': 7.2},
    'monopile': {
        'type': 'Monopile',
        'length': 69,
        'diameter': 7.2,
        'deck_space': 496.8,
        'weight': 800
    },
    
    'transition_piece': {
        'type': 'Transition Piece',
        'deck_space': 100,
        'weight': 400
    },
    
    'scour_protection_design': {
        'cost_per_tonne': 40,
    },
    
    # Electrical
    'array_system_design': {
        'cables': ['XLPE_185mm_33kV', 'XLPE_500mm_33kV']
    },
    
    'array_system': {
        'strategy': 'lay_bury'
    },
    
    'export_system': {
        'strategy': 'lay_bury'
    },
    
    'export_system_design': {
        'cables': 'XLPE_500mm_132kV',
        'percent_added_length': .2
    },
    
#     "offshore_substation_topside": {
#         "type": "Topside",
#         "deck_space": 200,
#         "weight": 2000,
#     },
#     "offshore_substation_substructure": {
#         "type": "Monopile",
#         "deck_space": 500,
#         "weight": 1850,
#         "length": 69,  # Assumed to be the same as monopile length
#     },


    # Phase specific configurations
    'MonopileInstallation': {
        'wtiv': 'Benchmarking_WTIV_monopile',  
        
    },
    
    'TurbineInstallation': {
        'wtiv': 'Benchmarking_WTIV_turbine'
    },
    
    'OffshoreSubstationInstallation': {
        "num_feeders": 1,
        "feeder": "seajacks_scylla",
    },    
    
    # Phases
    'design_phases': [
#         "MonopileDesign",
#         'ScourProtectionDesign',
#         'ArraySystemDesign',
#         'ExportSystemDesign',
#         'OffshoreSubstationDesign'
    ],
    
    'install_phases': {
        'MonopileInstallation': '07/01/2016',         # Updated dates 
#         'ScourProtectionInstallation': '09/01/2016',  # Placed at the end of the monopile installation
        'TurbineInstallation': '07/01/2016',
#         'ArrayCableInstallation': '06/29/2016',
#         'ExportCableInstallation': '03/17/2016',
#         'OffshoreSubstationInstallation': '08/01/2016'
    }
}

In [5]:
path = os.path.join(os.getcwd(), "library")
project = ProjectManager(config, weather=weather, library_path=path)

ORBIT library intialized at 'C:\Users\mshields\Documents\Analysis Tools\NREL\ORBIT\ORBIT-dev\dudgeon\library'


In [6]:
project.run_project()

In [35]:
# project._phases['TurbineInstallation'].wtiv.trip_data
project.project_dataframe.loc[(project.project_dataframe['phase']=='MonopileInstallation')]

,action,agent,cost,duration,level,location,message,phase,target,time,type
8,FastenItem,WTIV,2.500000e+05,12.000000,INFO,Port,NaN,MonopileInstallation,Monopile,84.000000,Operations
9,FastenItem,WTIV,1.666667e+05,8.000000,INFO,Port,NaN,MonopileInstallation,Transition Piece,92.000000,Operations
10,FastenItem,WTIV,2.500000e+05,12.000000,INFO,Port,NaN,MonopileInstallation,Monopile,104.000000,Operations
11,FastenItem,WTIV,1.666667e+05,8.000000,INFO,Port,NaN,MonopileInstallation,Transition Piece,112.000000,Operations
12,FastenItem,WTIV,2.500000e+05,12.000000,INFO,Port,NaN,MonopileInstallation,Monopile,124.000000,Operations
13,FastenItem,WTIV,1.666667e+05,8.000000,INFO,Port,NaN,MonopileInstallation,Transition Piece,132.000000,Operations
14,FastenItem,WTIV,2.500000e+05,12.000000,INFO,Port,NaN,MonopileInstallation,Monopile,144.000000,Operations
15,FastenItem,WTIV,1.666667e+05,8.000000,INFO,Port,NaN,MonopileInstallation,Transition Piece,152.000000,Operations
16,FastenItem,WTIV,2.500000e+05,12.000000,INFO,Port,NaN,MonopileInstallation,Monopile,164.000000,Operations
17,FastenItem,WTIV,1.666667e+05,8.000000,INFO,Port,NaN,MonopileInstallation,Transition Piece,172.000000,Operations


### Results
#### Array System Design

In [ ]:
project._phases["ArraySystemDesign"].total_cable_length_by_type

In [ ]:
project._phases['ArraySystemDesign'].num_full_strings

In [ ]:
project._phases['ArraySystemDesign'].num_partial_strings

In [ ]:
# Module outputs
project._phases['ArraySystemDesign'].total_phase_cost

In [ ]:
total = sum([v for k, v in project._phases['ArraySystemDesign'].total_cable_length_by_type.items()])
print(f"Total length: {total}")

In [ ]:
cables = project._phases['ArraySystemDesign'].cables

total_mass = sum([
    cables[k].linear_density*v for k, v in project._phases['ArraySystemDesign'].total_cable_length_by_type.items()
])
total_mass

#### Export System Design

In [ ]:
project._phases['ExportSystemDesign'].total_mass

In [ ]:
project._phases['ExportSystemDesign'].num_cables

In [ ]:
project._phases['ExportSystemDesign'].cable.cable_power

In [ ]:
project._phases['ExportSystemDesign']._plant_capacity

In [ ]:
project._phases['ExportSystemDesign'].total_phase_cost * 2/3

In [ ]:
project._phases['ExportSystemDesign'].total_length*2/3

#### Offshore Substation Design

In [ ]:
project._phases["OffshoreSubstationDesign"].total_phase_cost

In [ ]:
project._phases["OffshoreSubstationDesign"].design_result

#### Monopile Design

In [ ]:
# config['monopile']['weight'] * config['plant']['num_turbines'] * 3000 <<<<------ OLD


In [ ]:
project._phases["MonopileDesign"].total_phase_cost # <<<---- NEW

In [ ]:
project._phases["MonopileDesign"].design_result


#### Scour Protection Design

In [ ]:
project._phases['ScourProtectionDesign'].design_result  # <--- NEW, revised protection depth

In [ ]:
project._phases['ScourProtectionDesign'].protection_depth

In [ ]:
project._phases['ScourProtectionDesign'].total_phase_cost # <---- NEW

#### Monopile Installation

In [ ]:
project._phases['MonopileInstallation'].total_phase_time / (8760 / 12)

In [ ]:
project._phases['MonopileInstallation'].total_phase_cost

In [ ]:
project._phases['MonopileInstallation'].detailed_output

In [ ]:
project._phases['MonopileInstallation'].total_phase_time / 67

In [ ]:
mi = project.project_dataframe.loc[project.project_dataframe['phase']=='MonopileInstallation']

mi.groupby('action').sum()['duration']

#### Scour Protection Installation

In [ ]:
project._phases['ScourProtectionInstallation'].total_phase_time / (8760 / 12) # <--- NEW, revised protection depth

In [ ]:
project._phases['ScourProtectionInstallation'].total_phase_cost # <--- NEW, revised protection depth

In [ ]:
project._phases['ScourProtectionInstallation'].detailed_output

In [ ]:
project._phases['ScourProtectionInstallation'].total_phase_time / 67  # <--- NEW, revised protection depth

In [ ]:
spi = project.project_dataframe.loc[project.project_dataframe['phase']=='ScourProtectionInstallation']

spi.groupby('action').sum()['duration']

#### Turbine Installation

In [ ]:
project._phases['TurbineInstallation'].total_phase_time / (8760 / 12)  # NEW, turbine installation revised to eliminate too many crane reequips between blade installations

In [ ]:
project._phases['TurbineInstallation'].total_phase_cost  # NEW, see above

In [ ]:
project._phases['TurbineInstallation'].detailed_output

In [ ]:
project._phases['TurbineInstallation'].total_phase_time / 67  # NEW, see above

In [ ]:
ti = project.project_dataframe.loc[project.project_dataframe['phase']=='TurbineInstallation']

ti.groupby('action').sum()['duration']

In [ ]:
project._phases['TurbineInstallation'].wtiv.trip_data

#### Array System Installation

In [ ]:
project._phases['ArrayCableInstallation'].total_phase_time / (8760 / 12)  # NEW, not sure what changed but Rob should know. Burial rates?

In [ ]:
project._phases['ArrayCableInstallation'].total_phase_cost

In [ ]:
project._phases['ArrayCableInstallation'].detailed_output

In [ ]:
asi = project.project_dataframe.loc[project.project_dataframe['phase']=='ArrayCableInstallation']

asi.groupby('action').sum()['duration']

#### Export System Installation

In [ ]:
project._phases['ExportCableInstallation'].total_phase_time # NEW, not sure what changed but Rob should know. Burial rates?

In [ ]:
project._phases['ExportCableInstallation'].total_phase_cost # NEW, not sure what changed but Rob should know. Burial rates?

In [ ]:
project._phases['ExportCableInstallation'].detailed_output

In [ ]:
esi = project.project_dataframe.loc[project.project_dataframe['phase']=='ExportCableInstallation']

esi.groupby('action').sum()['duration']

#### Offshore Substation Installation

In [ ]:
project._phases['OffshoreSubstationInstallation'].total_phase_time / (24)

In [ ]:
project._phases['OffshoreSubstationInstallation'].total_phase_cost

In [ ]:
project._phases['OffshoreSubstationInstallation'].detailed_output

In [ ]:
osi = project.project_dataframe.loc[project.project_dataframe['phase']=='OffshoreSubstationInstallation']

osi.groupby('action').sum()['duration']

In [ ]:
project.project_dataframe["phase"].unique()

In [ ]:
project.design_results

In [ ]:
project.project_dataframe.time.max() / (8760 / 12)

In [ ]:
fig = plt.figure(figsize=(6, 4), dpi=200)
axis = fig.add_subplot(111)

pd.Series(project.phase_costs).plot(kind='bar', ax=axis)

axis.set_ylabel('Phase Cost ($USD)')

In [ ]:
fig = plt.figure(figsize=(6, 4), dpi=200)
axis = fig.add_subplot(111)

pd.Series(project.phase_times).plot(kind='bar', ax=axis)

axis.set_ylabel('Phase Time (h)')

In [ ]:
phases = project.phase_dates.keys()
_start_dates = [project.phase_dates[p]['start'] for p in phases]
start_floats = [plt_dates.datestr2num(i) for i in _start_dates]
_end_dates = [project.phase_dates[p]['end'] for p in phases]
end_floats = [plt_dates.datestr2num(i) for i in _end_dates]

In [ ]:
data = pd.DataFrame(columns=phases, data=[start_floats, end_floats], index=['Start', 'End']).T
data = data.reset_index().rename(columns={'index': 'Phase'})
data['Type'] = 'Model'

validation_data = data.copy()
validation_data = validation_data.set_index(keys='Phase')
validation_data.loc['MonopileInstallation', 'End'] = plt_dates.datestr2num("08/03/16 00:00")
validation_data.loc['ScourProtectionInstallation', 'End'] = plt_dates.datestr2num("09/30/16 00:00")
validation_data.loc['TurbineInstallation', 'End'] = plt_dates.datestr2num("09/04/17 00:00")
validation_data.loc['ArrayCableInstallation', 'End'] = plt_dates.datestr2num("08/27/16 00:00")
validation_data.loc['ExportCableInstallation', 'End'] = plt_dates.datestr2num("04/21/16 00:00")
validation_data['Type'] = 'Dudgeon'
validation_data = validation_data.reset_index().rename(columns={'index': 'Phase'})

data = pd.concat([data, validation_data])
data['Time'] = data['End'] - data['Start']
data

In [ ]:
import seaborn as sns
fig = plt.figure(figsize=(6, 4), dpi=200)
axis = fig.add_subplot(111)

axis = sns.barplot(x='Phase', y='Time', hue='Type', data=data, ax=axis)

for tick in axis.get_xticklabels():
    tick.set_rotation(90)
    
# axis.yaxis_date()
# fig.autofmt_xdate()

In [ ]:
fig = plt.figure(figsize=(6, 4), dpi=200)
axis = fig.add_subplot(111)

axis = sns.barplot(x='End', y='Phase', data=data, ax=axis, zorder=3, hue='Type')
axis = sns.barplot(x='Start', y='Phase', data=data, ax=axis, palette={'Model': 'w', 'Dudgeon': 'w'}, zorder=5, hue='Type')

axis.set_xlim(data['Start'].min()-10, data['End'].max()+10)

# Legend
handles, labels = axis.get_legend_handles_labels()
axis.legend(handles[:2], labels[:2], fontsize=8)

# Formatting
axis.set_xlabel('')
# axis.grid(axis='x', which='major', lw=0.5, zorder=4)
# Tick parameters
axis.tick_params(axis='both', labelsize=8, width=0.5, length=3)

# Plot formatting
for ax in ['top', 'bottom', 'left', 'right']:
    axis.spines[ax].set_linewidth(0.5)
    axis.spines[ax].set_zorder(20)

axis.xaxis_date()
fig.autofmt_xdate()